In [1]:
# import requests
# from basePy import get_access_token,open_pic2base64
# from ImageEnhancementPy import *

In [2]:
# 基础类

# coding: utf-8

# In[ ]:


import requests
import base64
import tkinter.filedialog
import json

def get_access_token(client_id, client_secret):
    # client_id 为官网获取的AK， client_secret 为官网获取的SK
    # 帮助文档
    # https://ai.baidu.com/docs#/Auth/top
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=' + client_id + '&client_secret=' + client_secret
    header = {'Content-Type': 'application/json; charset=UTF-8'}
    response1 = requests.post(url=host, headers=header)  # <class 'requests.models.Response'>
    json1 = response1.json()  # <class 'dict'>
    access_token = json1['access_token']
    print(access_token)
    return access_token


def open_pic2base64():
    # 本地图片地址，根据自己的图片进行修改
    # 打开本地图片，并转化为base64
    root = tkinter.Tk()  # 创建一个Tkinter.Tk()实例
    root.withdraw()  # 将Tkinter.Tk()实例隐藏
    file_path = tkinter.filedialog.askopenfilename(title=u'选择文件')
    f = open(file_path, 'rb')
    #84化
    img = base64.b64encode(f.read()).decode('utf-8')
    return img

import sys
import json

IS_PY3 = sys.version_info.major == 3
if IS_PY3:
    from urllib.request import urlopen
    from urllib.request import Request
    from urllib.error import URLError
    from urllib.parse import urlencode
    from urllib.parse import quote_plus
else:
    import urllib2
    from urllib import quote_plus
    from urllib2 import urlopen
    from urllib2 import Request
    from urllib2 import URLError
    from urllib import urlencode

# API_KEY = '4E1BG9lTnlSeIf1NQFlrSq6h'
# SECRET_KEY = '544ca4657ba8002e3dea3ac2f5fdd241'

# TEXT = "欢迎使用百度语音合成"

# 发音人选择, 基础音库：0为度小美，1为度小宇，3为度逍遥，4为度丫丫，
# 精品音库：5为度小娇，103为度米朵，106为度博文，110为度小童，111为度小萌，默认为度小美 
PER = 1
# 语速，取值0-15，默认为5中语速
SPD = 5
# 音调，取值0-15，默认为5中语调
PIT = 5
# 音量，取值0-9，默认为5中音量
VOL = 5
# 下载的文件格式, 3：mp3(default) 4： pcm-16k 5： pcm-8k 6. wav
AUE = 3

FORMATS = {3: "mp3", 4: "pcm", 5: "pcm", 6: "wav"}
FORMAT = FORMATS[AUE]

# CUID = "123456PYTHON"

TTS_URL = 'http://tsn.baidu.com/text2audio'


class DemoError(Exception):
    pass


"""  TOKEN start """

TOKEN_URL = 'http://openapi.baidu.com/oauth/2.0/token'
SCOPE = 'audio_tts_post'  # 有此scope表示有tts能力，没有请在网页里勾选


def fetch_token(client_id, client_secret):
    print("fetch token begin")
    params = {'grant_type': 'client_credentials',
              'client_id': client_id,
              'client_secret': client_secret}
    post_data = urlencode(params)
    if (IS_PY3):
        post_data = post_data.encode('utf-8')
    req = Request(TOKEN_URL, post_data)
    try:
        f = urlopen(req, timeout=5)
        result_str = f.read()
    except URLError as err:
        print('token http response http code : ' + str(err.code))
        result_str = err.read()
    if (IS_PY3):
        result_str = result_str.decode()

    print(result_str)
    result = json.loads(result_str)
    print(result)
    if ('access_token' in result.keys() and 'scope' in result.keys()):
        if not SCOPE in result['scope'].split(' '):
            raise DemoError('scope is not correct')
        print('SUCCESS WITH TOKEN: %s ; EXPIRES IN SECONDS: %s' % (result['access_token'], result['expires_in']))
        return result['access_token']
    else:
        raise DemoError('MAYBE API_KEY or SECRET_KEY not correct: access_token or scope not found in token response')


"""  TOKEN end """

def SynThesis(client_id, client_secret):
#     ak = '4E1BG9lTnlSeIf1NQFlrSq6h'
#     sk = '544ca4657ba8002e3dea3ac2f5fdd241'
#     client_id = ak
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
#     client_secret = sk
    token = fetch_token(client_id, client_secret)
    TEXT = input('''
    请输入需要合成语音的文本--
    ''')
    tex = quote_plus(TEXT)  # 此处TEXT需要两次urlencode
    print(tex)
    PER=input('''
    发音人选择, 基础音库：0为度小美，1为度小宇，3为度逍遥，4为度丫丫，
   精品音库：5为度小娇，103为度米朵，106为度博文，110为度小童，111为度小萌，默认为度小美--
    ''')
    SPD=input('''
    语速，取值0-15，默认为5中语速--
    ''')
    PIT=input('''
    音调，取值0-15，默认为5中语调--
    ''')
    VOL=input('''
    音量，取值0-9，默认为5中音量--
    ''')
    AUE=input('''
    下载的文件格式, 3：mp3(default) 4： pcm-16k 5： pcm-8k --
    ''')
    RESULT=input('''
    输出音频文件的名称 --
    ''')
    CUID = "123456PYTHON"
    params = {'tok': token, 'tex': tex, 'per': PER, 'spd': SPD, 
              'pit': PIT, 'vol': VOL, 'aue': AUE, 'cuid': CUID,
              'lan': 'zh', 'ctp': 1}  # lan ctp 固定参数

    data = urlencode(params)
    print('test on Web Browser' + TTS_URL + '?' + data)

    req = Request(TTS_URL, data.encode('utf-8'))
    has_error = False
    try:
        f = urlopen(req)
        result_str = f.read()

        headers = dict((name.lower(), value) for name, value in f.headers.items())

        has_error = ('content-type' not in headers.keys() or headers['content-type'].find('audio/') < 0)
    except  URLError as err:
        print('asr http response http code : ' + str(err.code))
        result_str = err.read()
        has_error = True
    list_l=[('3', "mp3"), ('4', "pcm-16k"), ('5', "pcm-8k")]
    FORMATS =dict(list_l)
#     {3: "mp3", 4: "pcm", 5: "pcm", 6: "wav"}
    FORMAT = FORMATS[AUE]
#     save_file = "error.txt" if has_error else 'result.' + FORMAT
    save_file = "error.txt" if has_error else RESULT+'.' + FORMAT
    with open(save_file, 'wb') as of:
        of.write(result_str)

    if has_error:
        if (IS_PY3):
            result_str = str(result_str, 'utf-8')
        print("tts api  error:" + result_str)

    print("result saved as :" + save_file)

# list = [('PER':"人名"), ('LOC':"地点"), ('ORG':"机构"),('TIME',"时间")]
# dic = dict(list)
# dic
DICT_1 = dict(PER = "人名", LOC = "地名", ORG ="机构",TIME="时间")
# DICT_1
DICT_2 = dict(n = "普通名词", nr = "人名", nz ="其他专名",a="形容词", u ="助词",xc="其他虚词",
             m = "数量词", c = "连词", s ="方位名词",ns="地名",nt="机构团体名",nw="作品名",
            v= "普通动词", vd = "动副词", vn ="名动词",t="时间",r = "代词", p ="介词",w="标点符号",
             ad = "副形词", an = "名形词", d ="副词",q="量词")

DICT_3 = dict(Ag="形语素",g="语素",ns="地名",u = "助词",
              a="形容词",h ="前接成分",nt="机构团体名",vg="动语素",
              ad = "副形词", i="成语", nz ="其他专名",v= "动词",
              an = "名形词", j="简略语", o= "拟声词", vd = "动副词",
              b="区分词",k = "后接成分",p ="介词",vn ="名动词",
              c = "连词",l ="习惯用语",q="量词", w="标点符号",
              dg ="副语素",m = "数量词",r = "代词",x="非语素字",
              d ="副词",Ng = "名语素",s ="处所词",y="语气词",
              e = "叹词", n = "普通名词",tg="时间语素",z="状态词",
              f ="方位词",nr = "人名", t="时间词", un ="未知词"
              )
######

DICT_4 = dict(ATT = "定中关系", QUN = "数量关系", COO ="并列关系",APP="同位关系", ADJ ="附加关系",VOB="动宾关系",
             POB = "介宾关系", SBV = "主谓关系", SIM ="比拟关系",TMP="时间关系",LOC="处所关系",DE="的结构"
              ,DI="地结构",DEI="得结构",SUO="所结构",BA="把结构",BEI="被结构",ADV="状中结构",CMP="动补结构",
            DBL= "兼语结构", CNJ = "关联词",  CS ="关联结构",MT="语态结构",VV = "连谓结构", HED ="核心结构",FOB="前置宾语",
             DOB = "双宾语", TOP = "主题", IS ="独立结构",IC="独立分词",DC = "依存分句", VNV = "叠词关系", YGC ="一个词",WP="标点")
# DICT_2
# {'bar': 3, 'egg': 2, 'spam': 1}
# {'bar': 3, 'egg': 2, 'spam': 1}
# DICT_1= {'PER':'人名', 'LOC':'地点', 'ORG':'机构','TIME','时间'}
# DICT_2= {'n':"普通名词", 'nr':"人名", 'nz':"专有地名",'a',"形容词"}

# 图像效果增强

## 黑白图片上色

In [20]:
def ColouRize(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/colourize"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='黑白上色.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))




## 去雾

In [24]:
def DeHaze(client_id, client_secret):
    # 去雾功能
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/dehaze"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='去雾.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

## 对比度

In [29]:
def Contrast_Enhance(client_id, client_secret):
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/contrast_enhance"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='对比度.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

## 无损放大

In [31]:
def Image_Quality_Enhance(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/image_quality_enhance"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='无损放大.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

## 拉伸恢复

In [35]:
def Stretch_Restore(client_id, client_secret):
  
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/stretch_restore"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='拉伸恢复.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

### 接下来的都是需要开启权限的

# 风格变化

In [49]:
def Style_Trans(client_id, client_secret,s):
    
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/style_trans"
    #这个是相关参数和open值
    header = {'Content-Type': 'application/json'}
    if s=='卡通风格':
        params = {"image": open_pic2base64(),"option":"cartoon"}
    if s=='铅笔风格':
#     if s=="铅笔风格"：
        params = {"image": open_pic2base64(),"option":"pencil"}
    if s=='彩色铅笔风格':
#     if s=="彩色铅笔风格"：
       params = {"image": open_pic2base64(),"option":"color_pencil"}
    if s=='彩色糖块风格':
#     if s=="彩色糖块风格"：
       params = {"image": open_pic2base64(),"option":"warm"}
    if s=='神奈川冲浪里油画风格':
#     if s=="神奈川冲浪里油画风格"：
       params = {"image": open_pic2base64(),"option":"wave"}
    if s=='奇异油画风格':
#     if s=="奇异油画风格"：
       params = {"image": open_pic2base64(),"option":"mononoke"}
    if s=='呐喊油画风格':
#     if s=="呐喊油画风格"：
       params = {"image": open_pic2base64(),"option":"scream"}
    if s=='哥特油画风格':
#     if s=="哥特油画风格"：
       params = {"image": open_pic2base64(),"option":"gothic"}
       
       
    
    name=s+".png"
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     name='风格卡通.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

In [70]:
#### 这个失败了我也不知道怎么做

def InpaInting(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/inpainting"
    #这个是相关参数和open值
#     {\"inpainting_type\":\"rectangle\",\"rectangle\":[{\"width\":92,\"top\":95,\"height\":36,\"left\":543}],\"image\":\"图片base64编码\"}"
    params = {"inpainting_type":"rectangle","rectangle":[{'width': 92, 'top': 25, 'height': 36, 'left': 543}],"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
    print(json1)
    name='修复.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

## 图像清晰化

In [66]:
def Image_Definition_Enhance(client_id, client_secret): 
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/image_definition_enhance"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='图像清晰化.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

## 动漫化

In [72]:
def Selfie_Anime(client_id, client_secret):
 
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/selfie_anime"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='动漫化.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

## 天空分割

In [7]:
def Sky_Seg(client_id, client_secret):

# 可智能分割出天空边界位置，输出天空和其余背景的灰度图和二值图
# 可用于图像二次处理，进行天空替换、抠图等图片编辑场景。

    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-process/v1/sky_seg"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    name='天空分割二值图.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["labelmap"]))
    name='天空分割灰度图.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["scoremap"]))

# 图像识别

In [31]:
# 通用物体和场景识别高级版
def Advanced_General(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v2/advanced_general"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    length=json1["result_num"]
#     print(length)
    for num in range(0,length): 
        print('**********************')
        print("上层标签", json1["result"][num]["root"])
        print("关键词", json1["result"][num]["keyword"])
        print('***********************')
#     print("年龄为", json1["result"]["face_list"][0]['age'], '岁')
#     print("人种为", json1["result"]["face_list"][0]['race']['type'])
#     print("颜值评分为", json1["result"]["face_list"][0]['beauty'], '分/100分')
#     print("是否带眼镜", json1["result"]["face_list"][0]['glasses']['type'])

## 图像单主体检测

In [32]:
def Object_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/object_detect"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
    print(json1)
#     print("标记", json1["person_info"][0]["body_parts"])
    print("定位位置的长方形的宽度", json1["result"]["width"])
    print("定位位置的长方形的高度", json1["result"]["height"])
    print("定位位置的长方形左上顶点的水平坐标", json1["result"]["left"])
    print("定位位置的长方形左上顶点的垂直坐标", json1["result"]["top"])

## 动物识别

In [37]:
def Animal_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/animal"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
    print(json1)
#     print("标记", json1["person_info"][0]["body_parts"])

#原来这个才是字典  0-1-2
    print(type(json1["result"]))
    # 这才是列数
    print(len(json1["result"]))
    # 不可识别人类
    length=len(json1["result"])
#     print(length)
    for num in range(0,length): 
        print('*******')
        print("类型为", json1["result"][num]["name"])
        print('*******')

## 植物识别

In [43]:
def Plant_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/plant"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print(type(json1["result"]))
    # 这才是列数
    length=len(json1["result"])
    # 不可识别人类
    for num in range(0,length): 
        print('*******')
        print("类型为", json1["result"][num]["name"])
        print('*******')

## logo识别

In [45]:
def Logo_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v2/logo"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print(type(json1["result"]))
    # 这才是列数
    length=len(json1["result"])
    # 不可识别人类
    for num in range(0,length): 
        print('*******')
        print("类型为", json1["result"][num]["name"])
        print('*******')

## 果蔬识别

In [48]:
def Ingredient_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/classify/ingredient"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    length=len(json1["result"])
    # 不可识别人类
    for num in range(0,length): 
        print('*******')
        print("类型为", json1["result"][num]["name"])
        print('*******')

## 菜品识别

In [63]:
# def Dish_Detect(client_id, client_secret):
#     # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
#     # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
#     #open_pic2是相关数据的代码
#     request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v2/dish"
#     #这个是相关参数和open值
#     params = {"image": open_pic2base64()}
#     header = {'Content-Type': 'application/x-www-form-urlencoded'}

#     access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
# #     request_url = request_url + "?access_token=" + access_token
#     #access_token是相应token
#     request_url = request_url + "?access_token=" + access_token
#     #相关参数
#     response1 = requests.post(url=request_url, data=params, headers=header)
#     # JSON结果
#     json1 = response1.json()
#     print(json1)
#     print(type(json1["result"]))
#     # 这才是列数
#     print(len(json1["result"]))
# #     print("标记", json1["person_info"][0]["body_parts"])
#     print("菜品名称为", json1["result"][0]["name"])

###### 单品版本
    
# def Dish_Detect(client_id, client_secret):
#     # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
#     # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
#     #open_pic2是相关数据的代码
#     request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v2/dish"
#     #这个是相关参数和open值
#     params = {"image": open_pic2base64()}
#     header = {'Content-Type': 'application/x-www-form-urlencoded'}

#     access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
# #     request_url = request_url + "?access_token=" + access_token
#     #access_token是相应token
#     request_url = request_url + "?access_token=" + access_token
#     #相关参数
#     response1 = requests.post(url=request_url, data=params, headers=header)
#     # JSON结果
#     json1 = response1.json()
#     print(json1)
#     print(type(json1["result"]))
#     # 这才是列数
#     length=json1["result_num"]
# #     print("标记", json1["person_info"][0]["body_parts"])
#     for num in range(0,length): 
#         print('*******')
#         print("菜品名称为", json1["result"][num]["name"])
#         print('*******')
# #     print("菜品名称为", json1["result"][0]["name"])
# ####这个是可以用的。。。。阿西吧，啊啊啊啊

## 红酒识别

In [65]:
def RedWine_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/redwine"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print(type(json1["result"]))
    # 这才是列数
#     print(len(json1["result"]))
#     print("标记", json1["person_info"][0]["body_parts"])

#  太多字段了懒得一条条写了
# 不一定什么都会返回
    print("红酒类型为", json1["result"]["classifyByColor"])
    print("红酒名为", json1["result"]["wineNameCn"])
#     print("红酒英文为", json1["result"]["wineNameEn"])
    print("产地国为", json1["result"]["countryCn"])
#     print("红酒产地国英文为", json1["result"]["countryEn"])
    print("产地为", json1["result"]["countryCn"])
    print("产区为", json1["result"]["regionCn"])
    print("子产区为", json1["result"]["subRegionCn"])
    print("酒庄为", json1["result"]["wineryCn"])
    print("色泽为", json1["result"]["color"])
    print("葡萄种类为", json1["result"]["grapeCn"])
    print("品尝温度为", json1["result"]["tasteTemperature"])
    print("描述为", json1["result"]["description"])

In [71]:
def Currency_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/currency"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print(type(json1["result"]))
    # 这才是列数
#     print(len(json1["result"]))
#     print("标记", json1["person_info"][0]["body_parts"])
    print("货币名称为", json1["result"]["currencyName"])

## 地标识别

In [74]:
def Landmark_Detect(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/landmark"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print(type(json1["result"]))
    # 这才是列数
#     print(len(json1["result"]))
#     print("标记", json1["person_info"][0]["body_parts"])
    print("地标名称为", json1["result"]["landmark"])

# 人体识别

## 人体关键点识别

In [100]:
def Body_Analysis(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/body_analysis"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#    person_num
    print("人数", json1["person_num"])
    length=json1["person_num"]
#     length=len(json1["person_num"])
#     print(type(json1["person_info"][0]["body_parts"]))
#     d=json1["person_info"][0]["body_parts"]
    for num in range(0,length): 
        print('第'+str(num+1)+'个人')
        print('*******************************************')
        x=str(json1["person_info"][num]["body_parts"]["right_wrist"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_wrist"]["y"])
        print("右手腕(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_wrist"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_wrist"]["y"])
        print("左手腕(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["top_head"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["top_head"]["y"])
        print("头顶(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_eye"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_eye"]["y"])
        print("左眼(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_eye"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_eye"]["y"])
        print("右眼(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["nose"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["nose"]["y"])
        print("鼻子(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_ear"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_ear"]["y"])
        print("左耳(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_ear"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_ear"]["y"])
        print("右耳(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_mouth_corner"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_mouth_corner"]["y"])
        print("左嘴角(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_mouth_corner"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_mouth_corner"]["y"])
        print("右嘴角(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["neck"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["neck"]["y"])
        print("颈部(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_shoulder"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_shoulder"]["y"])
        print("左肩(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_shoulder"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_shoulder"]["y"])
        print("右肩(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_elbow"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_elbow"]["y"])
        print("左手肘(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_elbow"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_elbow"]["y"])
        print("右手肘(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_hip"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_hip"]["y"])
        print("左髋部(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_hip"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_hip"]["y"])
        print("右髋部(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_knee"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_knee"]["y"])
        print("左膝(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_knee"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_knee"]["y"])
        print("右膝(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["left_ankle"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["left_ankle"]["y"])
        print("左脚踝(x,y)"+"("+x+","+y+")")
        print('*******')
        print('*******')
        x=str(json1["person_info"][num]["body_parts"]["right_ankle"]["x"])
        y=str(json1["person_info"][num]["body_parts"]["right_ankle"]["y"])
        print("右脚踝(x,y)"+"("+x+","+y+")")
        print('*******************************************')
        
       
    

## 人体检测和属性识别

In [106]:
# 调用百度API完成人体检测和属性识别
def Body_Attr(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/body_attr"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    
    print("人数", json1["person_num"])
    length=json1["person_num"]
    """
    性别、年龄阶段、衣着（含类别/颜色）、是否戴帽子、
是否戴口罩、是否戴眼镜、是否背包、是否抽烟、是否使用手机、身体朝向等。
    """
    for num in range(0,length): 
        print('第'+str(num+1)+'个人')
        print('*******************************************')
        print("身体超向", json1["person_info"][num]["attributes"]["orientation"]["name"])
        print("性别", json1["person_info"][num]["attributes"]["gender"]["name"])
        print("年龄", json1["person_info"][num]["attributes"]["age"]["name"])
        print("上身衣服", json1["person_info"][num]["attributes"]["upper_wear"]["name"])
        print("下身衣服", json1["person_info"][num]["attributes"]["lower_wear"]["name"])
        print("上身颜色", json1["person_info"][num]["attributes"]["upper_color"]["name"])
        print("下身颜色", json1["person_info"][num]["attributes"]["lower_color"]["name"])
        print("上身服饰纹理", json1["person_info"][num]["attributes"]["upper_wear_texture"]["name"])
        print("背包", json1["person_info"][num]["attributes"]["bag"]["name"])
        print("上身服饰细分类", json1["person_info"][num]["attributes"]["upper_wear_fg"]["name"])
        print("是否戴帽子", json1["person_info"][num]["attributes"]["headwear"]["name"])
        print("是否戴口罩", json1["person_info"][num]["attributes"]["face_mask"]["name"])
        print("是否戴眼镜", json1["person_info"][num]["attributes"]["glasses"]["name"])
        print("是否撑伞", json1["person_info"][num]["attributes"]["umbrella"]["name"])
        print("是否使用手机", json1["person_info"][num]["attributes"]["cellphone"]["name"])
        print("是否吸烟", json1["person_info"][num]["attributes"]["smoke"]["name"])
        print("是否有手提物", json1["person_info"][num]["attributes"]["carrying_item"]["name"])
        print("交通工具", json1["person_info"][num]["attributes"]["vehicle"]["name"])
        print("上方截断", json1["person_info"][num]["attributes"]["upper_cut"]["name"])
        print("下方截断", json1["person_info"][num]["attributes"]["lower_cut"]["name"])
        print("遮挡", json1["person_info"][num]["attributes"]["occlusion"]["name"])
        print("是否是正常人体", json1["person_info"][num]["attributes"]["is_human"]["name"])
        print('*******************************************')

## 人流量统计

In [109]:
'''
对于输入的一张图片（可正常解码，且长宽比适宜），识别和统计图像当中的人体个数（静态统计，不支持追踪和去重）。

适用于3米以上的中远距离俯拍，以头部为主要识别目标统计人数，无需正脸、全身照，
适应各类人流密集场景（如：机场、车展、景区、广场等）；
默认识别整图中的人数，支持指定不规则区域的人数统计，同时可输出渲染图片。
'''
def Body_Num(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/body_num"
    #这个是相关参数和open值 返回渲染图
    params = {"image": open_pic2base64(),"show":"true"}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print("标记", json1["image"])
    print("人数", json1["person_num"])
    #竟然没得
    name='A人流量渲染后的图.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["image"]))

## 手势识别

In [111]:
def GesTure(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/gesture"
    #这个是相关参数和open值 返回渲染图
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print("标记", json1["image"])
    print("手势", json1["result"][0]["classname"])

## 人像分割

In [114]:
def Body_Seg(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/body_seg"
    #这个是相关参数和open值 返回渲染图
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    
    # 二值图是黑的啊。。。。
    name='A二值图1.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["labelmap"]))
    name='A灰度图1.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["scoremap"]))
    name='A前景人像图_透明背景_1.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["foreground"]))

## 驾驶行为分析

In [166]:
def Driver_Behavior(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/driver_behavior"
    #这个是相关参数和open值 返回渲染图
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    
    # 二值图是黑的啊。。。。
#     name='A二值图1.png'
#     with open(name,'wb') as f:
#         f.write(base64.b64decode(json1["labelmap"]))
#     name='A灰度图1.png'
#     with open(name,'wb') as f:
#         f.write(base64.b64decode(json1["scoremap"]))
#     name='A前景人像图_透明背景_1.png'
#     with open(name,'wb') as f:
#         f.write(base64.b64decode(json1["foreground"]))
        
    print("总人数", json1["person_num"])
    print("驾驶人数", json1["driver_num"])
#     print(type(json1["person_info"][0]["attributes"]))
    dict_demo=json1["person_info"][0]["attributes"]
    list_1=[]
    for key in dict_demo:
        print(str(key)+','+str(dict_demo[key]))
        list_1.append(dict_demo[key]["score"])
    ######这个就到时候再说吧，我太懒了
    max_score=max(list_1)
    
    print("最高分数",max_score)
    # 总是空的我能怎么办。。。。
    print(get_keys(dict_demo,max_score))
#     for key,value in dict_demo.items():
#         if(value == max(dict_demo.values())):
#             print(key,value)
    
#     print(values[len(values)-1])
#     print('当前司机行为',max(dict_demo,key=dict_demo.get))
    
# 根据键值判断行为。。。。    
def get_keys(d, value):
    return [k for k,v in d.items() if v == value]
#     print("行为属性", json1["person_info"][0]["attributes"])


## 手部关键点

In [172]:
def Hand_Analysis(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/hand_analysis"
    #这个是相关参数和open值 返回渲染图
    img=open_pic2base64()
    params ={"image":img}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
    dict_demo=json1["hand_info"][0]["hand_parts"]
#     print(json1)
    for key in dict_demo:
        print(str(key)+','+str(dict_demo[key])) # 这里要加个排序

## 指尖检测

In [174]:
def FingerTip(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/fingertip"
    #这个是相关参数和open值 返回渲染图
    img=open_pic2base64()
    params ={"image":img}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    dict_demo=json1["hand_info"][0]["hand_parts"]
#     print(json1)
    for key in dict_demo:
        print(str(key)+','+str(dict_demo[key])) # 这里要加个排序

# 文字识别

## 通用文字识别

In [180]:
def General_basic_wenzi(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic"
    #这个是相关参数和open值
    params = {"image": open_pic2base64(),"language_type	":"CHN_ENG"}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    length=len(json1["words_result"])
    for num in range(0,length):
        print("文字为", json1["words_result"][num]["words"])

In [182]:
def General_wenzi(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general"
    #这个是相关参数和open值
    params = {"image": open_pic2base64(),"language_type	":"CHN_ENG"}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    length=len(json1["words_result"])
    for num in range(0,length):
        print("文字为", json1["words_result"][num]["words"])

In [184]:
def Accurate_accurate_wenzi(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/accurate"
    #这个是相关参数和open值
    params = {"image": open_pic2base64(),"language_type	":"CHN_ENG"}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    length=len(json1["words_result"])
    for num in range(0,length):
        print("文字为", json1["words_result"][num]["words"])

## 卡证文字识别

In [190]:
def Idcard_wenzi(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/idcard"
    #这个是相关参数和open值
    params = {"image": open_pic2base64(),"id_card_side":"front"}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print("文字为", json1["risk_type"])
#     print(json1['words_result'])
    print('住址',json1['words_result']['住址']['words'])
    print('出生地',json1['words_result']['出生']['words'])
    print('姓名',json1['words_result']['姓名']['words'])
    print('身份证号',json1['words_result']['公民身份号码']['words'])
    print('性别',json1['words_result']['性别']['words'])
    print('民族',json1['words_result']['民族']['words'])

In [193]:
def BankCard(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/bankcard"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
    print(json1)
#     print("文字为", json1["risk_type"])
    print('银行类型',json1['result']['bank_card_number'])
    print('银行卡号',json1['result']['bank_name'])

In [198]:
def Business_License(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/business_license"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("单位名称为", json1["words_result"]["单位名称"]["words"])
    print("类型为", json1["words_result"]["类型"]["words"])
    print("法人为", json1["words_result"]["法人"]["words"])
    print("地址为", json1["words_result"]["地址"]["words"])
    print("有效期为", json1["words_result"]["有效期"]["words"])
    print("证件编号为", json1["words_result"]["证件编号"]["words"])
    print("社会信用代码为", json1["words_result"]["社会信用代码"]["words"])

In [201]:
def Business_Card(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = " https://aip.baidubce.com/rest/2.0/ocr/v1/business_card"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("地址为", json1["words_result"]["ADDR"])
    print("传真为", json1["words_result"]["FAX"])
    print("电话为", json1["words_result"]["MOBILE"])
    print("PC为", json1["words_result"]["PC"])
    print("网址为", json1["words_result"]["URL"])
    print("固定电话为", json1["words_result"]["TEL"])
    print("公司为", json1["words_result"]["COMPANY"])
    # 没有识别到
    print("姓名为", json1["words_result"]["NAME"])
    # 难以识别
    print("职位为", json1["words_result"]["TITLE"])

In [205]:
def PassPort(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/passport"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("国家码为", json1["words_result"]["国家码"]["words"])
    print("护照签发地点为", json1["words_result"]["护照签发地点"]["words"])
    print("有效期至为", json1["words_result"]["有效期至"]["words"])
    print("签发机关为", json1["words_result"]["签发机关"]["words"])
    print("护照号码为", json1["words_result"]["护照号码"]["words"])
    print("姓名拼音为", json1["words_result"]["姓名拼音"]["words"])
    print("出生地点为", json1["words_result"]["出生地点"]["words"])
    # 没有识别到
    print("姓名为", json1["words_result"]["姓名"]["words"])
    # 难以识别
    print("签发日期为", json1["words_result"]["签发日期"]["words"])
    print("生日为", json1["words_result"]["生日"]["words"])
    # 难以识别
    print("性别为", json1["words_result"]["性别"]["words"])

In [206]:
def HK_Macau_exitentrypermit(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/HK_Macau_exitentrypermit"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("户籍为", json1["words_result"]["Address"]["words"])
    print("出生日期为", json1["words_result"]["Birthday"]["words"])
    print("性别为", json1["words_result"]["Sex"]["words"])
    print("卡号为", json1["words_result"]["CardNum"]["words"])
    print("中文名为", json1["words_result"]["NameChn"]["words"])
    print("英文名为", json1["words_result"]["NameEng"]["words"])
#     print("出生地点为", json1["words_result"]["出生地点"]["words"])
#     # 没有识别到
    print("有效日期为", json1["words_result"]["ValidDate"]["words"])

In [210]:
def Taiwan_Exitentrypermit(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/taiwan_exitentrypermit"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
    print(json1)
    print("户籍为", json1["words_result"]["Address"]["words"])
    print("出生日期为", json1["words_result"]["Birthday"]["words"])
    print("性别为", json1["words_result"]["Sex"]["words"])
    print("卡号为", json1["words_result"]["CardNum"]["words"])
    print("中文名为", json1["words_result"]["NameChn"]["words"])
    print("英文名为", json1["words_result"]["NameEng"]["words"])
#     print("出生地点为", json1["words_result"]["出生地点"]["words"])
#     # 没有识别到
    print("有效日期为", json1["words_result"]["ValidDate"]["words"])

In [212]:
def Household_Register(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/household_register"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("出生地址为", json1["words_result"]["BirthAddress"]["words"])
    print("出生日期为", json1["words_result"]["Birthday"]["words"])
    print("性别为", json1["words_result"]["Sex"]["words"])
    print("卡号为", json1["words_result"]["CardNo"]["words"])
    print("名字为", json1["words_result"]["Name"]["words"])
    print("民族为", json1["words_result"]["Nation"]["words"])
#     print("出生地点为", json1["words_result"]["出生地点"]["words"])
#     # 没有识别到
    print("与户主关系为", json1["words_result"]["Relationship"]["words"])

### 火车票

In [215]:
def Train_Ticket(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/train_ticket"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("日期为", json1["words_result"]["date"])
    print("车票号为", json1["words_result"]["ticket_num"])
    print("始发站为", json1["words_result"]["starting_station"])
    print("到达站为", json1["words_result"]["destination_station"])
    print("车次号为", json1["words_result"]["train_num"])
    print("车票金额为", json1["words_result"]["ticket_rates"])
    print("席别为", json1["words_result"]["seat_category"])
    print("乘客姓名", json1["words_result"]["name"])

### 驾驶证

In [ ]:
def Driving_License(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/driving_license"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("证号为", json1["words_result"]["证号"]["words"])
    print("有效期限", json1["words_result"]["有效期限"]["words"])
    print("准驾车型为", json1["words_result"]["准驾车型"]["words"])    
    print("有效起始日期为", json1["words_result"]["有效起始日期"]["words"])
    print("住址为", json1["words_result"]["住址"]["words"])
    print("姓名为", json1["words_result"]["姓名"]["words"])
    print("国籍为", json1["words_result"]["国籍"]["words"])
    print("出生日期为", json1["words_result"]["出生日期"]["words"])
    print("性别为", json1["words_result"]["性别"]["words"])
    print("初次领证日期为", json1["words_result"]["初次领证日期"]["words"])

In [217]:
def License_Plate(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/license_plate"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("车牌颜色", json1["words_result"]["color"])
    print("车牌号码", json1["words_result"]["number"])

In [219]:
def HandWriting(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/handwriting"
    #这个是相关参数和open值
    params = {"image": open_pic2base64()}
    header = {'Content-Type': 'application/x-www-form-urlencoded'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print(len(json1["words_result"]))
    length=len(json1["words_result"])
#     for i in  range(0,length)： 
    for num in range(0,length):
        print("手写文字", json1["words_result"][num]["words"])

# 人脸识别

In [222]:
def bd_rec_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"
    #这个是相关参数和open值
    params = {"image": open_pic2base64(), "image_type": "BASE64",
              "face_field": "age,beauty,glasses,gender,race"}
    header = {'Content-Type': 'application/json'}

    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=header)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("性别为", json1["result"]["face_list"][0]['gender']['type'])
    print("年龄为", json1["result"]["face_list"][0]['age'], '岁')
    print("人种为", json1["result"]["face_list"][0]['race']['type'])
    print("颜值评分为", json1["result"]["face_list"][0]['beauty'], '分/100分')
    print("是否带眼镜", json1["result"]["face_list"][0]['glasses']['type'])

In [226]:
import json
def bd_match_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/match"
    #这个是相关参数和open值
#     params= "[{\"image\": open_pic2base64(), \"image_type\": \"BASE64\",
#     \"face_type\": \"LIVE\", \"quality_control\": \"LOW\"},
#     {\"image\": open_pic2base64(), \"image_type\": \"BASE64\", 
#      \"face_type\": \"IDCARD\", \"quality_control\": \"LOW\"}]"
#      params = "[{\"image\": \"sfasq35sadvsvqwr5q...\", \"image_type\": \"BASE64\", \"face_type\": \"LIVE\", \"quality_control\": \"LOW\"},
#  {\"image\": \"sfasq35sadvsvqwr5q...\", \"image_type\": \"BASE64\", \"face_type\": \"IDCARD\", \"quality_control\": \"LOW\"}]"
# ]"
#     "[{"image": open_pic2base64(), "image_type": "BASE64","face_type": "LIVE"},{"image": open_pic2base64(), "image_type": "BASE64",  "face_type": "LIVE"}]"
    
#     header = {'Content-Type': 'application/json'}
    params=json.dumps([                          # 将字典数据转化为字符串 
        {"image":open_pic2base64(),"image_type":'BASE64',"face_type":"LIVE"},
        {"image":open_pic2base64(),"image_type":'BASE64',"face_type":"IDCARD"}
    ])
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
#     print("是一个人的概率",json1[]);
    print("是一个人的概率", json1["result"]["score"])

## 这一块关于人脸库的添加删除什么的，太懒了不想再写界面，我就加着吧。。

## 人脸库管理

### 人脸添加

In [ ]:
#人脸添加
"""
Created on 2019-06-21
@author: DaDaBaoBaoRen
"""

# 还算的上不错的功能
import requests
import base64
import tkinter.filedialog
import json

def get_access_token(client_id, client_secret):
    # client_id 为官网获取的AK， client_secret 为官网获取的SK
    # 帮助文档
    # https://ai.baidu.com/docs#/Auth/top
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=' + client_id + '&client_secret=' + client_secret
    header = {'Content-Type': 'application/json; charset=UTF-8'}
    response1 = requests.post(url=host, headers=header)  # <class 'requests.models.Response'>
    json1 = response1.json()  # <class 'dict'>
    access_token = json1['access_token']
    print(access_token)
    return access_token


def open_pic2base64():
    # 本地图片地址，根据自己的图片进行修改
    # 打开本地图片，并转化为base64
    root = tkinter.Tk()  # 创建一个Tkinter.Tk()实例
    root.withdraw()  # 将Tkinter.Tk()实例隐藏
    file_path = tkinter.filedialog.askopenfilename(title=u'选择文件')
    f = open(file_path, 'rb')
    #84化
    img = base64.b64encode(f.read()).decode('utf-8')
    return img

####这个是可以用的。。。。阿西吧，啊啊啊啊

def bd_add_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/user/add"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    params = {"image": open_pic2base64(), "image_type": "BASE64",
              "group_id": "demo1","user_id": "user2","user_info": "abc",
              "quality_control":"LOW","liveness_control":"NORMAL"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)
#     print("是一个人的概率",json1[]);
#     print("是一个人的概率", json1["result"]["score"])
#     print("年龄为", json1["result"]["face_list"][0]['age'], '岁')
#     print("人种为", json1["result"]["face_list"][0]['race']['type'])
#     print("颜值评分为", json1["result"]["face_list"][0]['beauty'], '分/100分')
#     print("是否带眼镜", json1["result"]["face_list"][0]['glasses']['type'])

if __name__ == '__main__':
    # 以下为代码功能测试：
    # 账户id，client_id 为官网获取的AK， client_secret 为官网获取的SK。
    # https://console.bce.baidu.com/ai/?fromai=1#/ai/face/app/list
    ak = 'W6n5m36xofq13wSEIQHFZeuQ'
    sk = 'XORa0FO3T7rdlT8X7WnDPPOUlWOHGoNN'
    client_id = ak
#     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
    client_secret = sk
#     'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'  

#z这他妈是怎么做到的我日我日
    # 实例1：人脸识别
    # 开始调用
    bd_add_face(client_id, client_secret)


### 人脸更新

In [ ]:
def bd_update_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/user/update"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    params = {"image": open_pic2base64(), "image_type": "BASE64",
              "group_id": "demo1","user_id": "user1","user_info": "abc",
              "quality_control":"LOW","liveness_control":"NORMAL"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

### 人脸删除

In [ ]:
def bd_delete_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/face/delete"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"group_id": "demo1","user_id": "user1","face_token": "6f645f67919e5c61c3c63945284d0415"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

### 人脸查询

In [ ]:
def bd_get_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/user/get"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"group_id": "demo1","user_id": "user1"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)
#     print("是一个人的概率",json1[]);
    print("相关信息", json1["result"]["user_list"])

#### 人脸列表

In [ ]:
def bd_getlist_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/face/getlist"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"group_id": "demo1","user_id": "user1"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

In [ ]:
### 用户列表

In [ ]:
def bd_getusers_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/group/getusers"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"group_id": "demo1"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

In [ ]:
### 复制人脸

In [ ]:
def bd_copy_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/user/copy"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"user_id": "user1","src_group_id": "demo1","dst_group_id": "group1"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

In [ ]:
### 创建组

In [ ]:
def bd_create_group_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/group/add"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"group_id": "tmp3"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

In [ ]:
### 删除组

In [ ]:
def bd_delete_group_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/group/delete"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"group_id": "tmp3"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

In [ ]:
### 获取组列表

In [ ]:
def bd_getlists_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceset/group/getlist"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    #删除对应的ID
    params = {"start":1 ,"length": 2}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

In [ ]:
### 人脸搜索

In [ ]:
def bd_search_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/search"
#     params = "{\"image\":\"027d8308a2ec665acb1bdf63e513bcb9\",\"image_type\":\"FACE_TOKEN\"
#     ,\"group_id_list\":
#     \"group_repeat,group_233\",\"quality_control\":\"LOW\",\"liveness_control\":\"NORMAL\"}"
    params ={"image": open_pic2base64(), "image_type": "BASE64",
              "group_id_list": "demo1","quality_control":"LOW","liveness_control":"NORMAL"}
#     headers = {'content-type': 'application/json'}
    headers = {'content-type': 'application/json'}
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

### 活体对比

In [ ]:
import json
def bd_faceverify_face(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/faceverify"
    #这个是相关参数和open值
#     params= "[{\"image\": open_pic2base64(), \"image_type\": \"BASE64\",
#     \"face_type\": \"LIVE\", \"quality_control\": \"LOW\"},
#     {\"image\": open_pic2base64(), \"image_type\": \"BASE64\", 
#      \"face_type\": \"IDCARD\", \"quality_control\": \"LOW\"}]"
#      params = "[{\"image\": \"sfasq35sadvsvqwr5q...\", \"image_type\": \"BASE64\", \"face_type\": \"LIVE\", \"quality_control\": \"LOW\"},
#  {\"image\": \"sfasq35sadvsvqwr5q...\", \"image_type\": \"BASE64\", \"face_type\": \"IDCARD\", \"quality_control\": \"LOW\"}]"
# ]"
#     "[{"image": open_pic2base64(), "image_type": "BASE64","face_type": "LIVE"},{"image": open_pic2base64(), "image_type": "BASE64",  "face_type": "LIVE"}]"
    
#     header = {'Content-Type': 'application/json'}
    params=json.dumps([                          # 将字典数据转化为字符串 
        {"image":open_pic2base64(),"image_type":'BASE64',"face_type":"LIVE"},
        {"image":open_pic2base64(),"image_type":'BASE64',"face_type":"IDCARD"}
    ])
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)
#     print("是一个人的概率",json1[]);
    print("活体概率", json1["result"]["face_liveness"])

In [ ]:
### 语音校验码

In [ ]:
def bd_session_code(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v1/faceliveness/sessioncode"
    #这个是相关参数和open值
#     params= "[{\"image\": open_pic2base64(), \"image_type\": \"BASE64\",
#     \"face_type\": \"LIVE\", \"quality_control\": \"LOW\"},
#     {\"image\": open_pic2base64(), \"image_type\": \"BASE64\", 
#      \"face_type\": \"IDCARD\", \"quality_control\": \"LOW\"}]"
#      params = "[{\"image\": \"sfasq35sadvsvqwr5q...\", \"image_type\": \"BASE64\", \"face_type\": \"LIVE\", \"quality_control\": \"LOW\"},
#  {\"image\": \"sfasq35sadvsvqwr5q...\", \"image_type\": \"BASE64\", \"face_type\": \"IDCARD\", \"quality_control\": \"LOW\"}]"
# ]"
#     "[{"image": open_pic2base64(), "image_type": "BASE64","face_type": "LIVE"},{"image": open_pic2base64(), "image_type": "BASE64",  "face_type": "LIVE"}]"
    
#     header = {'Content-Type': 'application/json'}
    params = {"min_code_length":4 ,"max_code_length": 5}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)

In [ ]:
#### 变老小男女

In [230]:
def bd_action_type(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v1/editattr"
 #小孩子
#     params = {"image":open_pic2base64() ,"image_type": "BASE64","action_type":"TO_KID","quality_control": "NONE"}
# 变老
#     params = {"image":open_pic2base64() ,"image_type": "BASE64","action_type":"TO_OLD","quality_control": "NONE"}
# 变男
    params = {"image":open_pic2base64() ,"image_type": "BASE64","action_type":"TO_MALE","quality_control": "NONE"}
# 变女
#     params = {"image":open_pic2base64() ,"image_type": "BASE64","action_type":"TO_FEMALE","quality_control": "NONE"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()

#     change_base64_as_img(json1["result"]["image"])  
        #### 这个可能是真的
    # 到时候加一个字符串连接就行
    name='haitongMale.png'
    with open(name,'wb') as f:
        f.write(base64.b64decode(json1["result"]["image"]))

In [232]:
### 痘斑痣检测 我忘了功能

In [237]:
def bd_acnespotmole_type(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v1/acnespotmole"
    params = {"image":open_pic2base64() ,"image_type": "BASE64","max_face_num": 5}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print("痘信息",json1["result"]["face_list"][0]["acne_list"])
    print("斑信息",json1["result"]["face_list"][0]["speckle_list"])
    print("痣信息",json1["result"]["face_list"][0]["mole_list"])
#     change_base64_as_img(json1["result"]["image"])  
        #### 这个可能是真的
    # 到时候加一个字符串连接就行
#     name='yingjunJiaZhi.png'
#     with open(name,'wb') as f:
#         f.write(base64.b64decode(json1["result"]["image"]))

In [240]:
def bd_skin_type(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v1/skin_color"
    params = {"image":open_pic2base64() ,"image_type": "BASE64"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print('肤色',json1["result"]["face_list"][0]["skin"]["color"])
#     change_base64_as_img(json1["result"]["image"])  
        #### 这个可能是真的
    # 到时候加一个字符串连接就行
#     name='yingjunJiaZhi.png'
#     with open(name,'wb') as f:
#         f.write(base64.b64decode(json1["result"]["image"]))

In [ ]:
### 皮肤光滑度

In [245]:
def bd_skin_smooth_type(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rest/2.0/face/v1/skin_smooth"
    params = {"image":open_pic2base64() ,"image_type": "BASE64"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

# 语音识别

In [252]:
## 普通的语音识别

In [ ]:
#### 效率贼低下

import requests
import json
import base64
import os
import logging
import speech_recognition as sr

#这个是源自老子调用的API
def get_token():
    logging.info('开始获取token...')
    #获取token
    baidu_server = "https://openapi.baidu.com/oauth/2.0/token?"
    grant_type = "client_credentials"
#     client_id = "up7sdaBHdk09sbMk1l6ijszx"
#     client_secret = "XmoFEcE4i8ErqBbnuSlgWb2B81AKXard"
    client_id = "RB9qhLaLK5QsAPj9c5E1gLkI"
    client_secret = "HKM7QogjMuuKtnqZkf2Q0FyA5XEk0Q3i"

    #拼url
    url = f"{baidu_server}grant_type={grant_type}&client_id={client_id}&client_secret={client_secret}"
    res = requests.post(url)
    token = json.loads(res.text)["access_token"]
    return token


def audio_baidu(filename):
    logging.info('开始识别语音文件...')
    with open(filename, "rb") as f:
        speech = base64.b64encode(f.read()).decode('utf-8')
    size = os.path.getsize(filename)
    token = get_token()
    headers = {'Content-Type': 'application/json'}
    url = "https://vop.baidu.com/server_api"
    data = {
        "format": "wav",
        "rate": "16000",
        "dev_pid": "1536",
        "speech": speech,
        "cuid": "TEDxPY",
        "len": size,
        "channel": 1,
        "token": token,
    }
    req = requests.post(url, json.dumps(data), headers)
    result = json.loads(req.text)
    print(result)
    print('1')
    if result["err_msg"] == "success.":
        print(result['result'])
        #我要在这里加入一个with open等等功能
        return result['result']
    else:
        print("内容获取失败，退出语音识别")
        return -1


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    wav_num = 0
    while True:
        r = sr.Recognizer()
        #启用麦克风
        mic = sr.Microphone()
        #这里应该是个连续不断的过程。到时候有的改
        logging.info('录音中...')
        with mic as source:
            #降噪
            r.adjust_for_ambient_noise(source)
            audio = r.listen(source)
        with open(f"yj00{wav_num}.wav", "wb") as f:
#         with open(f"H:\FFOutput\梅香如故DOWN.wav", "wb") as f:
            #将麦克风录到的声音保存为wav文件
            f.write(audio.get_wav_data(convert_rate=16000))
        logging.info('录音结束，识别中...')
        target = audio_baidu(f"yj00{wav_num}.wav")
#         target = audio_baidu(f"yj00{wav_num}.wav")
#         with open(f"H:\FFOutput\梅香如故DOWN.wav", "wb") as f:
#             #将麦克风录到的声音保存为wav文件
#             f.write(audio.get_wav_data(convert_rate=16000))
#         logging.info('录音结束，识别中...')
#         target = audio_baidu(f"H:\FFOutput\梅香如故DOWN.wav")
        if target == -1:
            break
        wav_num += 1


In [ ]:
### 版本2 通过读取位置
#!/usr/bin/python3
# -*- coding: utf-8 -*-
import json,base64,requests,urllib.request

class Speech_recognition():
    '''百度语音识别'''
    def key(self):
        #获取token秘钥
        body = {
            "grant_type"    : "client_credentials",
            "client_id"     : "RB9qhLaLK5QsAPj9c5E1gLkI",
            "client_secret" : "HKM7QogjMuuKtnqZkf2Q0FyA5XEk0Q3i"
        }
        url  = "https://aip.baidubce.com/oauth/2.0/token?"
        r = requests.post(url,data=body,verify=True,timeout=2)
        respond = json.loads(r.text)
        return  respond["access_token"]

    def api(self,audio_data):
        with open(audio_data,'rb') as fp:
            audio_data = fp.read()
        speech_data = base64.b64encode(audio_data).decode("utf-8")
        #用Base64编码具有不可读性，需要解码后才能阅读
        speech_length = len(audio_data)
        post_data = {"format":"wav","rate": 16000,
        "channel": 1,"cuid":"123456python",
        "token": self.key(), "speech": speech_data,"len": speech_length}

        json_data = json.dumps(post_data).encode("utf-8")
        json_length = len(json_data)

        try:
            req = urllib.request.Request("http://vop.baidu.com/server_api",data=json_data)
            req.add_header("Content-Type", "application/json")
            req.add_header("Content-Length", json_length)
            resp = urllib.request.urlopen(req,timeout=20)##合成时间不多，识别最耗时
            resp = resp.read()
            resp_data = json.loads(resp.decode("utf-8"))

        except: print('超时')

        if resp_data["err_no"] == 0:
            return resp_data["result"][0]
            #return resp_data["result"]  #返回识别出的文字
        else:
            return ''


if __name__ == '__main__':
    data = Speech_recognition().api(r"I:\NewC#demo\语音识别代码和wav\16k.wav")
    print("语音识别结果:",data)
    input("--")


In [253]:
## 语音合成

In [186]:
# coding=utf-8

##### ？？？？这是啥  竟然能用了！！！ 我日那我就不用那么累了，阿西吧
import sys
import json

IS_PY3 = sys.version_info.major == 3
if IS_PY3:
    from urllib.request import urlopen
    from urllib.request import Request
    from urllib.error import URLError
    from urllib.parse import urlencode
    from urllib.parse import quote_plus
else:
    import urllib2
    from urllib import quote_plus
    from urllib2 import urlopen
    from urllib2 import Request
    from urllib2 import URLError
    from urllib import urlencode

# API_KEY = '4E1BG9lTnlSeIf1NQFlrSq6h'
# SECRET_KEY = '544ca4657ba8002e3dea3ac2f5fdd241'

# TEXT = "欢迎使用百度语音合成"

# 发音人选择, 基础音库：0为度小美，1为度小宇，3为度逍遥，4为度丫丫，
# 精品音库：5为度小娇，103为度米朵，106为度博文，110为度小童，111为度小萌，默认为度小美 
PER = 1
# 语速，取值0-15，默认为5中语速
SPD = 5
# 音调，取值0-15，默认为5中语调
PIT = 5
# 音量，取值0-9，默认为5中音量
VOL = 5
# 下载的文件格式, 3：mp3(default) 4： pcm-16k 5： pcm-8k 6. wav
AUE = 3

FORMATS = {3: "mp3", 4: "pcm", 5: "pcm", 6: "wav"}
FORMAT = FORMATS[AUE]

# CUID = "123456PYTHON"

TTS_URL = 'http://tsn.baidu.com/text2audio'


class DemoError(Exception):
    pass


"""  TOKEN start """

TOKEN_URL = 'http://openapi.baidu.com/oauth/2.0/token'
SCOPE = 'audio_tts_post'  # 有此scope表示有tts能力，没有请在网页里勾选


def fetch_token(client_id, client_secret):
    print("fetch token begin")
    params = {'grant_type': 'client_credentials',
              'client_id': client_id,
              'client_secret': client_secret}
    post_data = urlencode(params)
    if (IS_PY3):
        post_data = post_data.encode('utf-8')
    req = Request(TOKEN_URL, post_data)
    try:
        f = urlopen(req, timeout=5)
        result_str = f.read()
    except URLError as err:
        print('token http response http code : ' + str(err.code))
        result_str = err.read()
    if (IS_PY3):
        result_str = result_str.decode()

    print(result_str)
    result = json.loads(result_str)
    print(result)
    if ('access_token' in result.keys() and 'scope' in result.keys()):
        if not SCOPE in result['scope'].split(' '):
            raise DemoError('scope is not correct')
        print('SUCCESS WITH TOKEN: %s ; EXPIRES IN SECONDS: %s' % (result['access_token'], result['expires_in']))
        return result['access_token']
    else:
        raise DemoError('MAYBE API_KEY or SECRET_KEY not correct: access_token or scope not found in token response')


"""  TOKEN end """

def SynThesis(client_id, client_secret):
#     ak = '4E1BG9lTnlSeIf1NQFlrSq6h'
#     sk = '544ca4657ba8002e3dea3ac2f5fdd241'
#     client_id = ak
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
#     client_secret = sk
    token = fetch_token(client_id, client_secret)
    TEXT = input('''
    请输入需要合成语音的文本--
    ''')
    tex = quote_plus(TEXT)  # 此处TEXT需要两次urlencode
    print(tex)
    PER=input('''
    发音人选择, 基础音库：0为度小美，1为度小宇，3为度逍遥，4为度丫丫，
   精品音库：5为度小娇，103为度米朵，106为度博文，110为度小童，111为度小萌，默认为度小美--
    ''')
    SPD=input('''
    语速，取值0-15，默认为5中语速--
    ''')
    PIT=input('''
    音调，取值0-15，默认为5中语调--
    ''')
    VOL=input('''
    音量，取值0-9，默认为5中音量--
    ''')
    AUE=input('''
    下载的文件格式, 3：mp3(default) 4： pcm-16k 5： pcm-8k --
    ''')
    RESULT=input('''
    输出音频文件的名称 --
    ''')
    CUID = "123456PYTHON"
    params = {'tok': token, 'tex': tex, 'per': PER, 'spd': SPD, 
              'pit': PIT, 'vol': VOL, 'aue': AUE, 'cuid': CUID,
              'lan': 'zh', 'ctp': 1}  # lan ctp 固定参数

    data = urlencode(params)
    print('test on Web Browser' + TTS_URL + '?' + data)

    req = Request(TTS_URL, data.encode('utf-8'))
    has_error = False
    try:
        f = urlopen(req)
        result_str = f.read()

        headers = dict((name.lower(), value) for name, value in f.headers.items())

        has_error = ('content-type' not in headers.keys() or headers['content-type'].find('audio/') < 0)
    except  URLError as err:
        print('asr http response http code : ' + str(err.code))
        result_str = err.read()
        has_error = True
    list_l=[('3', "mp3"), ('4', "pcm-16k"), ('5', "pcm-8k")]
    FORMATS =dict(list_l)
#     {3: "mp3", 4: "pcm", 5: "pcm", 6: "wav"}
    FORMAT = FORMATS[AUE]
#     save_file = "error.txt" if has_error else 'result.' + FORMAT
    save_file = "error.txt" if has_error else RESULT+'.' + FORMAT
    with open(save_file, 'wb') as of:
        of.write(result_str)

    if has_error:
        if (IS_PY3):
            result_str = str(result_str, 'utf-8')
        print("tts api  error:" + result_str)

    print("result saved as :" + save_file)
    

# if __name__ == '__main__':
#     ak = '4E1BG9lTnlSeIf1NQFlrSq6h'
#     sk = '544ca4657ba8002e3dea3ac2f5fdd241'
#     client_id = ak
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
#     client_secret = sk
#     SynThesis(client_id, client_secret)
#     token = fetch_token(client_id, client_secret)
#     TEXT = input('''
#     请输入需要合成语音的文本--
#     ''')
#     tex = quote_plus(TEXT)  # 此处TEXT需要两次urlencode
#     print(tex)
#     PER=input('''
#     发音人选择, 基础音库：0为度小美，1为度小宇，3为度逍遥，4为度丫丫，
#    精品音库：5为度小娇，103为度米朵，106为度博文，110为度小童，111为度小萌，默认为度小美--
#     ''')
#     SPD=input('''
#     语速，取值0-15，默认为5中语速--
#     ''')
#     PIT=input('''
#     音调，取值0-15，默认为5中语调--
#     ''')
#     VOL=input('''
#     音量，取值0-9，默认为5中音量--
#     ''')
#     AUE=input('''
#     下载的文件格式, 3：mp3(default) 4： pcm-16k 5： pcm-8k --
#     ''')
#     RESULT=input('''
#     输出音频文件的名称 --
#     ''')
#     CUID = "123456PYTHON"
#     params = {'tok': token, 'tex': tex, 'per': PER, 'spd': SPD, 
#               'pit': PIT, 'vol': VOL, 'aue': AUE, 'cuid': CUID,
#               'lan': 'zh', 'ctp': 1}  # lan ctp 固定参数

#     data = urlencode(params)
#     print('test on Web Browser' + TTS_URL + '?' + data)

#     req = Request(TTS_URL, data.encode('utf-8'))
#     has_error = False
#     try:
#         f = urlopen(req)
#         result_str = f.read()

#         headers = dict((name.lower(), value) for name, value in f.headers.items())

#         has_error = ('content-type' not in headers.keys() or headers['content-type'].find('audio/') < 0)
#     except  URLError as err:
#         print('asr http response http code : ' + str(err.code))
#         result_str = err.read()
#         has_error = True
#     list_l=[('3', "mp3"), ('4', "pcm-16k"), ('5', "pcm-8k")]
#     FORMATS =dict(list_l)
# #     {3: "mp3", 4: "pcm", 5: "pcm", 6: "wav"}
#     FORMAT = FORMATS[AUE]
# #     save_file = "error.txt" if has_error else 'result.' + FORMAT
#     save_file = "error.txt" if has_error else RESULT+'.' + FORMAT
#     with open(save_file, 'wb') as of:
#         of.write(result_str)

#     if has_error:
#         if (IS_PY3):
#             result_str = str(result_str, 'utf-8')
#         print("tts api  error:" + result_str)

#     print("result saved as :" + save_file)


fetch token begin
{"access_token":"24.65e53e1e988201eb9d9c2330b5f355f3.2592000.1588586312.282335-10854623","session_key":"9mzdDcOq+E9K3kz2Rs14yxWdK0oL7l9cTOctxZzLHz2YUcnQuL5kmWYT\/DaSNfaXwxyxH4EjyjzhyNTMJ9M\/eGB70cVNDw==","scope":"brain_speech_realtime vis-faceverify_FACE_Police brain_enhanced_asr unit_\u7406\u89e3\u4e0e\u4ea4\u4e92V2 public audio_voice_assistant_get audio_tts_post wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify smartapp_opensource_openapi smartapp_opensource_recapi qatest_scope1 fake_face_detect_\u5f00\u653eScope vis-ocr_\u865a\u62df\u4eba\u7269\u52a9\u7406 idl-video_\u865a\u62df\u4eba\u7269\u52a9\u7406","refresh_token":"25.8e192b703380504aca67c48c0cd27a9a.315360000.1901354312.282335-10854623","session_secre

# 自然语言处理

In [2]:
import json
def LeXer(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/lexer"
    s=input("--")
#     s="百度牛逼"
    params =json.dumps({'text': s})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [72]:
import json
def LeXer(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/lexer"
    s=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'text': s})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    length=len(json1["items"])
    for num in range(0,length):
        print("*******************"+str(num+1)+"****************")  
        print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
#         age=int(input('请输入年龄'))
        if json1["items"][num]["ne"]=='':
            print(str("ne：") + '无')
        else:
            print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
        if json1["items"][num]["pos"]=='':
            print(str("pos：") + '无')
        else:
            print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
        print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [74]:
# import json
def Lexer_Custom(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/lexer_custom"
    s=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'text': s})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    length=len(json1["items"])
    for num in range(0,length):
        print("*******************"+str(num+1)+"****************")  
        print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
#         age=int(input('请输入年龄'))
        if json1["items"][num]["ne"]=='':
            print(str("ne：") + '无')
        else:
            print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
        if json1["items"][num]["pos"]=='':
            print(str("pos：") + '无')
        else:
            print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
        print("***********************************")
#     print(json1["items"])
#     for num in range(0,length):
#         print(str(d[num]["basic_words"])+':'+str(d[num]["pos"]))
#         print(str(key) + ':' +str(d[key]))
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [98]:
# import json
def DepParser(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/depparser"
    s=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'text': s})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)
    length=len(json1["items"])
    for num in range(0,length):
        print("*******************"+str(num+1)+"****************")  
        print(str("word") + ':' + str(json1["items"][num]["word"]))
#         age=int(input('请输入年龄'))
        if json1["items"][num]["postag"]=='':
            print(str("postag：") + '无')
        else:
            print(str("postag：") + ':' + str(DICT_3[json1["items"][num]["postag"]]))
        if json1["items"][num]["deprel"]=='':
            print(str("deprel：") + '无')
        else:
            print(str("deprel：") + ':' + str(DICT_4[json1["items"][num]["deprel"]]))
        print("***********************************")
#     print(json1["items"])
#     for num in range(0,length):
#         print(str(d[num]["basic_words"])+':'+str(d[num]["pos"]))
#         print(str(key) + ':' +str(d[key]))
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [88]:
# import json
def WordEmbedding(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/word_emb_vec"
    s=input("请输入需要分析的文本--")
#     s2=input("请输入需要分析的文本")
#     s="百度牛逼"
    params =json.dumps({'word': s})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    word_list=[]
#     print(json1)
    length=len(json1["vec"])
#     length=len(json1["items"])
    for num in range(0,length):
        print(json1["vec"][num])
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print(json1["items"])
#     for num in range(0,length):
#         print(str(d[num]["basic_words"])+':'+str(d[num]["pos"]))
#         print(str(key) + ':' +str(d[key]))
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [90]:
# import json
def Dnnlm(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/dnnlm_cn"
    s=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'text': s})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print(json1)
    print("句子通顺度",str(json1["ppl"]))
#     length=len(json1["items"])
#     for num in range(0,length):
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print(json1["items"])
#     for num in range(0,length):
#         print(str(d[num]["basic_words"])+':'+str(d[num]["pos"]))
#         print(str(key) + ':' +str(d[key]))
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [100]:
import json
def SimNet(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/simnet"
    s=input("请输入需要分析的文本--")
    s1=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'text_1': s,'text_2': s1})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print("文本相似度",str(json1["score"]))
#     print(json1)
#     length=len(json1["items"])
#     for num in range(0,length):
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [102]:
import json
def KeyWord(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/keyword"
    s=input("请输入需要分析的文本--")
    s1=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'title': s,'content': s1})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print("文本相似度",str(json1["score"]))
#     print(json1)
    length=len(json1["items"])
    for num in range(0,length):
        print("*******************"+str(num+1)+"****************")  
        print(str("tag：") + ':' + str(json1["items"][num]["tag"]))
        print(str("score：") + ':'+ str(json1["items"][num]["score"]))
        print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [114]:
import json
def ToPic(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/topic"
    s=input("请输入需要分析的文本--")
    s1=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'title': s,'content': s1})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    # 二次分类结果
    length=len(json1["item"]["lv2_tag_list"])
#     print('一次分类结果')
#     print("*******************"+str(num+1)+"****************")  
#         print(str("tag：") + ':' + str(json1["items"]["lv1_tag_list"][0]["tag"]))
#         print(str("score：") + ':'+ str(json1["items"]["lv1_tag_list"][0]["score"]))
#         print("***********************************")
    print('一级分类结果')
    print(str("tag：") +':'+str(json1["item"]["lv2_tag_list"][0]["tag"]))
    print(str("score：")+':'+str(json1["item"]["lv2_tag_list"][0]["score"]))
    print('二次分类结果')
    for num in range(0,length):
        print("*******************"+str(num+1)+"****************")  
        print(str("tag：") + ':' + str(json1["item"]["lv2_tag_list"][num]["tag"]))
        print(str("score：") + ':'+ str(json1["item"]["lv2_tag_list"][num]["score"]))
        print("***********************************")
#         print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [118]:
import json
def EcNet(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/ecnet"
    s=input("请输入需要分析的文本--")
#     s1=input("请输入需要分析的文本--")
#     s="百度牛逼"
    params =json.dumps({'text': s})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print("正确结果",str(json1["item"]["correct_query"]))
#     print(json1)
#     length=len(json1["items"])
#     for num in range(0,length):
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [120]:
import json
def News_Summary(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/news_summary"
    s=input("请输入需要分析的标题--")
    s1=input("请输入需要分析的文本--")
    s2=input("请设定摘要的字数（必须是200-500内的数字，否则会报错）--")
#     s="百度牛逼"
    params =json.dumps({'title': s,'content': s1,'max_summary_len': s2})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
    print("摘要结果",str(json1["summary"]))
#     print(json1)
#     length=len(json1["items"])
#     for num in range(0,length):
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [146]:
import json
def Comment_Tag(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag"
    s=input("请输入需要分析的文本--")
    type='''
1	酒店	『酒店设备齐全、干净卫生』->『酒店设备齐全』、『干净卫生』
2	KTV	『环境一般般把，音响设备也一般，隔音太差』->『环境一般』、『音响设备一般』、『隔音差』
3	丽人	『手法专业，重要的是效果很棒』->『手法专业』、『效果不错』
4	美食餐饮	『但是味道太好啦，舍不得剩下』->『味道不错』
5	旅游	『景区交通方便，是不错的旅游景点』->『交通方便』、『旅游景点不错』
6	健康	『环境很棒，技师服务热情』->『环境不错』、『服务热情』
7	教育	『教学质量不错，老师很有经验』->『教学质量不错』、『老师有经验』
8	商业	『该公司服务好，收费低，效率高』->『服务好』、『收费低』、『效率高』
9	房产	『该房周围设施齐全、出行十分方便』->『设施齐全』、『出行方便』
10	汽车	『路宝的优点就是安全性能高、空间大』->『安全性能高』、『空间大』
11	生活	『速度挺快、服务态度也不错』->『速度快』、『服务好』
12	购物	『他家的东西还是挺贵的』->『消费贵』
    '''
#     13	3C	『手机待机时间长』->『待机时间长』
    s1=input("请输入评论行业类型例如"+type+"--")
#     s2=input("请设定摘要的字数（必须是200-500内的数字，否则会报错）--")
#     s="百度牛逼"
    params =json.dumps({'text': s,'type': s1})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print(json1["items"][0])
#     print(type(json1["items"][0]))
    flag=int(json1["items"][0]["sentiment"])
    if flag==0:
        print("评论情感","消极")
    if flag==1:
        print("评论情感","中性")
    if flag==2:
        print("评论情感","积极性")
    print("属性词",str(json1["items"][0]["prop"]))
    print("描述词",str(json1["items"][0]["adj"]))
    print("对应于该情感搭配的短句摘要",str(json1["items"][0]["abstract"]))
#     print("属性词",str(json1["items"]["prop"]))
#     print("描述词",str(json1["items"]["adj"]))
#     print("对应于该情感搭配的短句摘要",str(json1["items"]["abstract"]))
#     print(json1)
#     length=len(json1["items"])
#     for num in range(0,length):
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [153]:
import json
def Sentiment_Classify(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify"
    s=input("请输入需要分析的文本--")
#     type='''
# 1	酒店	『酒店设备齐全、干净卫生』->『酒店设备齐全』、『干净卫生』
# 2	KTV	『环境一般般把，音响设备也一般，隔音太差』->『环境一般』、『音响设备一般』、『隔音差』
# 3	丽人	『手法专业，重要的是效果很棒』->『手法专业』、『效果不错』
# 4	美食餐饮	『但是味道太好啦，舍不得剩下』->『味道不错』
# 5	旅游	『景区交通方便，是不错的旅游景点』->『交通方便』、『旅游景点不错』
# 6	健康	『环境很棒，技师服务热情』->『环境不错』、『服务热情』
# 7	教育	『教学质量不错，老师很有经验』->『教学质量不错』、『老师有经验』
# 8	商业	『该公司服务好，收费低，效率高』->『服务好』、『收费低』、『效率高』
# 9	房产	『该房周围设施齐全、出行十分方便』->『设施齐全』、『出行方便』
# 10	汽车	『路宝的优点就是安全性能高、空间大』->『安全性能高』、『空间大』
# 11	生活	『速度挺快、服务态度也不错』->『速度快』、『服务好』
# 12	购物	『他家的东西还是挺贵的』->『消费贵』
#     '''
#     13	3C	『手机待机时间长』->『待机时间长』
#     s1=input("请输入评论行业类型例如"+type+"--")
#     s2=input("请设定摘要的字数（必须是200-500内的数字，否则会报错）--")
#     s="百度牛逼"
#     params =json.dumps({'text': s})
    params =json.dumps({'text': s,'type': s1})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print(json1["items"][0])
#     print(type(json1["items"][0]))
    flag=int(json1["items"][0]["sentiment"])
    if flag==0:
        print("评论情感","消极")
    if flag==1:
        print("评论情感","中性")
    if flag==2:
        print("评论情感","积极性")
    
#     print(json1)
#     length=len(json1["items"])
#     for num in range(0,length):
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [171]:
import json
def EmoTion(client_id, client_secret):
    # 识别人脸，给出性别、年龄、人种、颜值分数、是否带眼镜等信息
    # 帮助文档中python代码基于python2,本文已经转换为python3x调试通过。
    
    #open_pic2是相关数据的代码
    request_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/emotion"
    s=input("请输入需要分析情感的文本--")
    type='''
    default（默认项-不区分场景），
    talk（闲聊对话-如度秘聊天等），
    task（任务型对话-如导航对话等），
    customer_service（客服对话-如电信/银行客服等）
    '''
#     type='''
# 1	酒店	『酒店设备齐全、干净卫生』->『酒店设备齐全』、『干净卫生』
# 2	KTV	『环境一般般把，音响设备也一般，隔音太差』->『环境一般』、『音响设备一般』、『隔音差』
# 3	丽人	『手法专业，重要的是效果很棒』->『手法专业』、『效果不错』
# 4	美食餐饮	『但是味道太好啦，舍不得剩下』->『味道不错』
# 5	旅游	『景区交通方便，是不错的旅游景点』->『交通方便』、『旅游景点不错』
# 6	健康	『环境很棒，技师服务热情』->『环境不错』、『服务热情』
# 7	教育	『教学质量不错，老师很有经验』->『教学质量不错』、『老师有经验』
# 8	商业	『该公司服务好，收费低，效率高』->『服务好』、『收费低』、『效率高』
# 9	房产	『该房周围设施齐全、出行十分方便』->『设施齐全』、『出行方便』
# 10	汽车	『路宝的优点就是安全性能高、空间大』->『安全性能高』、『空间大』
# 11	生活	『速度挺快、服务态度也不错』->『速度快』、『服务好』
# 12	购物	『他家的东西还是挺贵的』->『消费贵』
#     '''
#     13	3C	『手机待机时间长』->『待机时间长』
    s1=input("请输入对话场景例如"+type+"--")
#     s2=input("请设定摘要的字数（必须是200-500内的数字，否则会报错）--")
#     s="百度牛逼"
    params =json.dumps({'text': s,'scene': s1})
#     {"text": "百度是一家高科技公司"}
    headers = {'content-type': 'application/json'}
    
    access_token = get_access_token(client_id, client_secret)  # '[调用鉴权接口获取的token]'
#     request_url = request_url + "?access_token=" + access_token
    #access_token是相应token
    request_url = request_url + "?access_token=" + access_token
    #相关参数
    response1 = requests.post(url=request_url, data=params, headers=headers)
    # JSON结果
    json1 = response1.json()
#     print(json1)
    print(json1["items"][0])
#     print(type(json1["items"][0]))
    flag=json1["items"][0]["label"]
    if flag=='pessimistic':
        print("对话情感","消极")
    if flag=='neutral':
        print("对话情感","中性")
    if flag=='optimistic':
        print("对话情感","积极")
        
#     flag1=json1["items"][0]["subitems"]["label"]
#     if flag=='pessimistic':
#         print("对话情感","消极，哈哈哈哈，你不开心朕就高兴")
#     if flag=='neutral':
#         print("对话情感","中性,朕已阅")
#     if flag=='optimistic':
#         print("对话情感","积极，关我屁事")
    print(json1["items"][0]["replies"])
#     print(type(json1["items"][0]))
#     print(json1)
#     length=len(json1["items"])
#     for num in range(0,length):
#         print("*******************"+str(num+1)+"****************")  
#         print(str("basic_words") + ':' + str(json1["items"][num]["basic_words"]))
# #         age=int(input('请输入年龄'))
#         if json1["items"][num]["ne"]=='':
#             print(str("ne：") + '无')
#         else:
#             print(str("ne：") + ':' + str(DICT_1[json1["items"][num]["ne"]]))
#         if json1["items"][num]["pos"]=='':
#             print(str("pos：") + '无')
#         else:
#             print(str("pos：") + ':' + str(DICT_2[json1["items"][num]["pos"]]))
#         print("***********************************")
#     print('光滑度',json1["result"]["face_list"][0]["skin"]["smooth"])

In [ ]:
if __name__ == '__main__':
    # 以下为代码功能测试：
    # 账户id，client_id 为官网获取的AK， client_secret 为官网获取的SK。
    # https://console.bce.baidu.com/ai/?fromai=1#/ai/face/app/list
    
    
####### 图像增强#########
    ak = ''
    sk = ''
#     ColouRize(client_id, client_secret)
#     DeHaze(client_id, client_secret)
#     Contrast_Enhance(client_id, client_secret)
#     Image_Quality_Enhance(client_id, client_secret)
#     Stretch_Restore(client_id, client_secret)
#     Style_Trans(client_id, client_secret,'哥特油画风格')
#     InpaInting(client_id, client_secret)  #失败的。。。。
#     Image_Definition_Enhance(client_id, client_secret)
#     Selfie_Anime(client_id, client_secret)  # 垃圾一般
#     Sky_Seg(client_id, client_secret)  # 我也不知道有啥用  百度API输出二值图也有问题，另一个API也有问题
#     client_id = ak
#     client_secret = sk


####### 图像识别#########
    ak = ''
    sk = ''
#     client_id = ak
#     client_secret = sk
#     Advanced_General(client_id, client_secret)
#     Object_Detect(client_id, client_secret)
#     Animal_Detect(client_id, client_secret)
#     Plant_Detect(client_id, client_secret)
#     Logo_Detect(client_id, client_secret)
#     Ingredient_Detect(client_id, client_secret)
#     Dish_Search(client_id, client_secret)   #### 有的时候会发生未初始化
#     RedWine_Detect(client_id, client_secret)
#     Currency_Detect(client_id, client_secret)
#     Landmark_Detect(client_id, client_secret)
    
    
    
####### 人体分析##########
    ak = ''
    sk = ''
#     client_id = ak
#     client_secret = sk
#     #如果有问题
# #     Body_Analysis(client_id, client_secret)
# #     Body_Attr(client_id, client_secret)
# #     Body_Num(client_id, client_secret)
# #     GesTure(client_id, client_secret)
# #     Body_Seg(client_id, client_secret)
# #     Driver_Behavior(client_id, client_secret)  # 懒得比较了 这里需要进行一个根据键值取得键对的操作，但我不会
# #     Hand_Analysis(client_id, client_secret) # 请允许老夫懒惰
#     FingerTip(client_id, client_secret)




####### 文字识别 #############
    ak = ''
    sk = ''
#     client_id = ak
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
#     client_secret = sk
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'  

# #z这他妈是怎么做到的我日我日
#     # 实例1：人脸识别
#     # 开始调用
# #     General_basic_wenzi(client_id, client_secret)
# #     Accurate_accurate_wenzi(client_id, client_secret)
# #     Idcard_wenzi(client_id, client_secret)
# #     BankCard(client_id, client_secret)
# #     Business_License(client_id, client_secret)
# #     PassPort(client_id, client_secret)
# #     HK_Macau_exitentrypermit(client_id, client_secret)
# #     Taiwan_Exitentrypermit(client_id, client_secret)
# #     Household_Register(client_id, client_secret)
# #     Train_Ticket(client_id, client_secret)
# #     Driving_License(client_id, client_secret)  #涉及他人隐私不玩这个
# #     License_Plate(client_id, client_secret)
#     HandWriting(client_id, client_secret)



#########人脸识别################
    ak = ''
    sk = ''
#     client_id = ak
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
#     client_secret = sk
#     'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'  

#z这他妈是怎么做到的我日我日
    # 实例1：人脸识别
    # 开始调用
#     bd_rec_face(client_id, client_secret)
#     bd_match_face(client_id, client_secret)
#     bd_action_type(client_id, client_secret)
#     bd_acnespotmole_type(client_id, client_secret)
#     bd_skin_type(client_id, client_secret)
#     bd_skin_smooth_type(client_id, client_secret)
#     bd_gradient_type(client_id, client_secret)
#     bd_superclear_type(client_id, client_secret)
############################



#############语音识别########
    ak = ''
    sk = ''
#     client_id = ak
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
#     client_secret = sk
    # 百度语音识别非常垃圾，做好心理准备，只有加钱才能提升识别率！！！
    # 所以还是直接玩个语音合成吧
#     EmoTion(client_id, client_secret)
#     ak = ''
#     sk = ''
    client_id = ak
#     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
    client_secret = sk
    SynThesis(client_id, client_secret)
#     token = fetch_token(client_id, client_secret)
#     TEXT = input('''
#     请输入需要合成语音的文本--
#     ''')
#     tex = quote_plus(TEXT)  # 此处TEXT需要两次urlencode
#     print(tex)
#     PER=input('''
#     发音人选择, 基础音库：0为度小美，1为度小宇，3为度逍遥，4为度丫丫，
#    精品音库：5为度小娇，103为度米朵，106为度博文，110为度小童，111为度小萌，默认为度小美--
#     ''')
#     SPD=input('''
#     语速，取值0-15，默认为5中语速--
#     ''')
#     PIT=input('''
#     音调，取值0-15，默认为5中语调--
#     ''')
#     VOL=input('''
#     音量，取值0-9，默认为5中音量--
#     ''')
#     AUE=input('''
#     下载的文件格式, 3：mp3(default) 4： pcm-16k 5： pcm-8k --
#     ''')
#     RESULT=input('''
#     输出音频文件的名称 --
#     ''')
#     CUID = "123456PYTHON"
#     params = {'tok': token, 'tex': tex, 'per': PER, 'spd': SPD, 
#               'pit': PIT, 'vol': VOL, 'aue': AUE, 'cuid': CUID,
#               'lan': 'zh', 'ctp': 1}  # lan ctp 固定参数

#     data = urlencode(params)
#     print('test on Web Browser' + TTS_URL + '?' + data)

#     req = Request(TTS_URL, data.encode('utf-8'))
#     has_error = False
#     try:
#         f = urlopen(req)
#         result_str = f.read()

#         headers = dict((name.lower(), value) for name, value in f.headers.items())

#         has_error = ('content-type' not in headers.keys() or headers['content-type'].find('audio/') < 0)
#     except  URLError as err:
#         print('asr http response http code : ' + str(err.code))
#         result_str = err.read()
#         has_error = True
#     list_l=[('3', "mp3"), ('4', "pcm-16k"), ('5', "pcm-8k")]
#     FORMATS =dict(list_l)
# #     {3: "mp3", 4: "pcm", 5: "pcm", 6: "wav"}
#     FORMAT = FORMATS[AUE]
# #     save_file = "error.txt" if has_error else 'result.' + FORMAT
#     save_file = "error.txt" if has_error else RESULT+'.' + FORMAT
#     with open(save_file, 'wb') as of:
#         of.write(result_str)

#     if has_error:
#         if (IS_PY3):
#             result_str = str(result_str, 'utf-8')
#         print("tts api  error:" + result_str)

#     print("result saved as :" + save_file)



###### 自然语言处理  ######
#     ak = ''
#     sk = ''
#     client_id = ak
# #     'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
#     client_secret = sk
#     LeXer(client_id, client_secret)
#     Lexer_Custom(client_id, client_secret)
#     DepParser(client_id, client_secret)
    # 中间有问题是什么鬼呢？？？？
#     WordEmbedding(client_id, client_secret)
#     Dnnlm(client_id, client_secret)
#     SimNet(client_id, client_secret)
#     KeyWord(client_id, client_secret)
#     ToPic(client_id, client_secret)
#     EcNet(client_id, client_secret)
#     News_Summary(client_id, client_secret)
#     Comment_Tag(client_id, client_secret)
#     Sentiment_Classify(client_id, client_secret)
#     EmoTion(client_id, client_secret)  # 感觉这个功能很智障啊
#############################


fetch token begin
{"access_token":"24.a68c08755290081197daa2bdd447a10e.2592000.1588586923.282335-10854623","session_key":"9mzdWuq1OnDu1ZSXyAVBOiK\/TOap9IJd6rKPDpMxtMAqKGGvmrNImaFOthtBvMlfZ7JxO3vvbIs+IgNh+vvpERpZA+9MyQ==","scope":"brain_speech_realtime vis-faceverify_FACE_Police brain_enhanced_asr unit_\u7406\u89e3\u4e0e\u4ea4\u4e92V2 public audio_voice_assistant_get audio_tts_post wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify smartapp_opensource_openapi smartapp_opensource_recapi qatest_scope1 fake_face_detect_\u5f00\u653eScope vis-ocr_\u865a\u62df\u4eba\u7269\u52a9\u7406 idl-video_\u865a\u62df\u4eba\u7269\u52a9\u7406","refresh_token":"25.8b2b78afd97f91414e4248c50c684d16.315360000.1901354923.282335-10854623","session_secret